In [ ]:
%pip install transformers==4.28.0 #Installing latest version is running into dependency problems
%pip install datasets
%pip install sentencepiece
%pip install rouge_score
%pip install nbformat
%pip install plotly
%pip install torch

In [1]:
# funcao para fazer um print para mostra o texto em diferentes linhas
import textwrap
def print_lines(texto,w=50):
    lineas = textwrap.wrap(texto, width=w)
    for linea in lineas:
        print(linea)

In [2]:
from datasets import load_dataset,load_metric
# Load dataset from huggingface
dataset = load_dataset("exams", "crosslingual_with_para_pt")
dataset

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 740
    })
    validation: Dataset({
        features: ['id', 'question', 'answerKey', 'info'],
        num_rows: 184
    })
})

In [3]:
# Manipulate data set to build inputs
questions = [x['stem'] for x in dataset['train']['question']]
subjects = [x['subject'] for x in dataset['train']['info']]
choices = [x['choices'] for x in dataset['train']['question']]
correto = list()
for answerKey in dataset['train']['answerKey']:
    correto.append('Sim' if answerKey=='A' else 'Não')
    correto.append('Sim' if answerKey=='B' else 'Não')
    correto.append('Sim' if answerKey=='C' else 'Não')
    correto.append('Sim' if answerKey=='D' else 'Não')
subjects = [x['subject'] for x in dataset['train']['info']]
inputs = ["Tópico: {} Questão: {} Correto: {}".format(subjects[i//4], questions[i//4], correto[i]) for i in range(4*len(questions))]

In [4]:
# Manipulate dataset to build expected outputs
choices = [x['choices'] for x in dataset['train']['question']]
outputs = list()
for choice in choices:
    for i in range(4):
        outputs.append(choice['text'][i])

In [5]:
prefix = "question: "
max_input_length = 512
max_target_length = 32

In [6]:
def preprocess_examples(examples):
    """
    example: batch conjunto de exemplos
    Essa funcão é uma das mais importante já que formata nosso texto de entrada tokenizando o texto
    para esse finetunning usamos o dataset de um jeito diferente onde a entrada para o modelo sera uma string com tópico e questão
    e nossa saida esperada(outputs) serão alternativas.
    """
    questions = [x['stem'] for x in dataset['train']['question']]
    subjects = [x['subject'] for x in dataset['train']['info']]
    choices = [x['choices'] for x in dataset['train']['question']]
    correto = list()
    for answerKey in dataset['train']['answerKey']:
        correto.append('Sim' if answerKey=='A' else 'Não')
        correto.append('Sim' if answerKey=='B' else 'Não')
        correto.append('Sim' if answerKey=='C' else 'Não')
        correto.append('Sim' if answerKey=='D' else 'Não')
    subjects = [x['subject'] for x in dataset['train']['info']]
    inputs = ["Tópico: {} Questão: {} Correto: {}".format(subjects[i//4], questions[i//4], correto[i]) for i in range(4*len(questions))]
    outputs = list()
    for choice in choices:
        for i in range(4):
            outputs.append(choice['text'][i])
    # inputs e outputs sao texto agora eles vai ser tokenizados lembrar que model_inputs tem diferentes campos ver tutorial 1
    model_inputs = tokenizer(inputs, max_length=max_input_length, padding="max_length", truncation=True)
    # o label ficara só com os tokens_id
    labels = tokenizer(outputs, max_length=max_target_length, padding="max_length", truncation=True).input_ids
    labels_with_ignore_index = []
    for labels_example in labels:
        # devido ao processo de padding é importante não considerar esses ids
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)
    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs

In [7]:
from torch.utils.data import DataLoader
def create_dataloaders(train_batch_size=8, eval_batch_size=32):
    train_dataloader = DataLoader(encoded_train_ds, shuffle=True, batch_size=train_batch_size)
    val_dataloader = DataLoader(encoded_val_ds, shuffle=False, batch_size=eval_batch_size)

    return train_dataloader, val_dataloader

In [8]:
from transformers import T5Tokenizer
# modelo base para o finetunning
checkpoint = 'unicamp-dl/ptt5-base-portuguese-vocab'
# inicializa o tokenizer
tokenizer = T5Tokenizer.from_pretrained(checkpoint,model_max_length=max_input_length)

In [9]:
# separa o dataset
train,val = dataset['train'],dataset['validation']

In [10]:
encoded_train_ds = train.map(preprocess_examples, batched=True, remove_columns=train.column_names)
encoded_val_ds = val.map(preprocess_examples, batched=True, remove_columns=val.column_names)

In [11]:
encoded_train_ds.set_format(type="torch")
encoded_val_ds.set_format(type="torch")

In [12]:
from transformers import EarlyStoppingCallback
from transformers import T5Model, T5ForConditionalGeneration

# PyTorch
model_pt = T5ForConditionalGeneration.from_pretrained(checkpoint)
# early stopping e uma tecnica onde o finetuning para quando ele nao melhorar depois de 3 vezes consecutivas
early = EarlyStoppingCallback(early_stopping_patience=3)

In [13]:
import nltk
# essa biblioteca e importante para a funcao compute metrics
nltk.download('punkt')
metric = load_metric("rouge")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-13-d6866b6a5e99>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [14]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    # tem a predicoes e os labels(o que é esperado)
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # calculo das metricas
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [15]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pt)

In [16]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 20
# Show the training loss with every epoch
logging_steps = len(encoded_train_ds) // batch_size
# formatacao do modelo mudar se precisar para um padrao
model_name = "question2answer"
args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}",
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    # armazena só um modelo no colab é imporante não usar muita memoria
    save_total_limit=1,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    # garante que o modelo armazene o melhor modelo
    load_best_model_at_end=True,
    save_strategy="epoch",
    fp16=True
)

In [17]:
from transformers import Seq2SeqTrainer
# Criamos o Objeto trainer que vai receber os argumentos anterios juntos com as entradas numericas
trainer = Seq2SeqTrainer(
    model_pt,
    args,
    train_dataset=encoded_train_ds,
    eval_dataset=encoded_val_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early]
)

In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,4.399800,3.249607,14.950600,2.520000,13.780200,13.787200,9.304100
2,3.387300,2.745557,18.237400,4.408900,17.064400,17.081600,9.524300
3,3.027000,2.406184,20.671700,6.129200,19.332300,19.358700,9.845900
4,2.767100,2.138383,22.829400,7.822900,21.548000,21.558200,9.491900
5,2.543000,1.916799,24.619200,9.610300,23.298100,23.278600,9.996300
6,2.376900,1.738224,28.096800,12.259700,26.658600,26.658700,9.928700
7,2.228100,1.592981,30.722200,14.855100,29.158600,29.157900,9.991200
8,2.091500,1.465600,33.527800,17.362700,31.910500,31.886800,10.049700
9,1.993100,1.358796,35.331000,19.395900,33.676000,33.671700,10.290900
10,1.908100,1.277647,36.447700,20.451800,34.786100,34.787500,10.407800


TrainOutput(global_step=7400, training_loss=2.145211948704075, metrics={'train_runtime': 10817.2282, 'train_samples_per_second': 5.473, 'train_steps_per_second': 0.684, 'total_flos': 3.6050307121152e+16, 'train_loss': 2.145211948704075, 'epoch': 20.0})

In [19]:
model_pt.to("cpu")
model_pt.save_pretrained("temporal"+"_best/")

In [20]:
val

Dataset({
    features: ['id', 'question', 'answerKey', 'info'],
    num_rows: 184
})

In [21]:
val['question'][100]['choices']['para'][0]

'Banco Europeu de Investimento\n\nO Banco Europeu de Investimento (BEI), é instrumento financeiro da União Europeia. O Banco Mundial e o Banco Europeu de Investimento, por exemplo, só participam do financiamento de projetos no setor do carvão em casos excepcionais. Foi Vice-Presidente do Banco Europeu de Investimento entre 2007 e 2010. Desempenha ainda funções de administrador não executivo na Nutrinveste e no Banco Finantia, e de membro do Comité de Investimento da PVCI (Portugal Venture Capital Iniciative), um fundo de fundos de investimento ligado ao Banco Europeu de Investimento. A cidade do Luxemburgo contém várias instituições da União Europeia, incluindo o Tribunal de Justiça Europeu, o Tribunal de Contas e o Banco Europeu de Investimento. A cidade do Luxemburgo contém várias instituições da União Europeia, incluindo o Tribunal de Justiça Europeu, o Tribunal de Contas e o Banco Europeu de Investimento. A cidade do Luxemburgo contém várias instituições da União Europeia, incluind

In [22]:
val['question'][100]['stem']

'No processo de aprovação do orçamento comunitário intervém o'

In [23]:
val['question'][100]['choices']['text'][0]

'Banco Europeu de Investimento'

In [24]:
query_correct = ["Tópico: {} Questão: {} Correto: Sim".format(val['info'][100]['subject'], val['question'][100]['stem'])]
query_wrong = ["Tópico: {} Questão: {} Correto: Não".format(val['info'][100]['subject'], val['question'][100]['stem'])]

In [25]:
inputs_correct = tokenizer(query_correct, return_tensors="pt",padding=True,truncation=True).input_ids
inputs_wrong = tokenizer(query_wrong, return_tensors="pt",padding=True,truncation=True).input_ids

In [26]:
outputs_correct = model_pt.generate(inputs_correct,max_new_tokens=max_target_length,num_beams=10,num_return_sequences=10)
outputs_wrong = model_pt.generate(inputs_wrong,max_new_tokens=max_target_length,num_beams=10,num_return_sequences=10)

In [27]:
alternativas_corretas = tokenizer.batch_decode(outputs_correct, skip_special_tokens=True)
alternativas_erradas = tokenizer.batch_decode(outputs_wrong, skip_special_tokens=True)

In [28]:
for x in alternativas_corretas:
  print(x)

défice orçamental
Conselho de Estado-membro da União Europeia
défice do Estado-membro da União Europeia
Conselho de Estado-membro, que define a política fiscal do Estado-membro da União Europeia
Conselhos de Estado-membros
Conselho de Estado-membro, que define as prioridades económicas para o ano seguinte ao ano anterior ao ano anterior ao ano anterior ao ano anterior ao ano anterior
Conselho de Estado-membro, que define o plano de ação a ser seguido pelo Conselho de Estado-membro da União Europeia
Conselho de Estado-membros
Conselho de Estado-membro, que define o plano de ação a ser seguido pelo Estado-membro da União Europeia
Conselho de Estado-membro, que define as prioridades económicas para o ano seguinte à data da aprovação do orçamento


In [29]:
for x in alternativas_erradas:
  print(x)

défice orçamental
Conselho de Estado-membro da União Europeia
Tribunal de Contas do Estado-membro da União Europeia
Tribunal de Contas da União Europeia
Conselhos de Estado-membros
Conselho de Estado-membro, que define a política fiscal do Estado-membro da União Europeia
Conselho de Estado-membro, que faz parte do Conselho de Estado-membro da União Europeia
Conselho de Estado-membro, que define as prioridades económicas para cada Estado-membro da União Europeia
Conselho de Estado-membros
Conselho de Estado-membro, que define o plano de ação a ser seguido pelo Conselho de Estado-membro da União Europeia
